In [1]:
import sys
import os
import json
sys.path.append('..')

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Import modules
import src.ingest as ingest
from src.understand import analyze_meeting, print_analysis_results
from src.validate import validate_tasks, deduplicate_tasks
from src.planner import plan_tasks

from src.action import generate_followup_email, export_to_csv, export_to_json
from src.dashboard import generate_task_dashboard, display_task_summary

from src.evaluate import run_comprehensive_evaluation, print_evaluation_summary
from datetime import datetime

d:\Meeting-Execution-Agent\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print("🚀 COMPLETE MEETING PIPELINE TEST")
print("=" * 60)

# Step 1: Load and process transcript
print("📁 Step 1: Loading transcript...")
try:
    transcript_text = ingest.process_transcript('../data/sample_transcripts/meeting_01.txt')
    print(f"✅ Transcript loaded: {len(transcript_text)} characters")
    print("Preview (first 300 chars):")
    print(transcript_text[:300])
    print()
except Exception as e:
    print(f"❌ Error loading transcript: {e}")
    transcript_text = ""

🚀 COMPLETE MEETING PIPELINE TEST
📁 Step 1: Loading transcript...
✅ Transcript loaded: 6885 characters
Preview (first 300 chars):
Jeffy: Okay folks, let's get started. We have about 20 minutes. Mira, wanna kick us off with where we stand on the roadmap?

Mira: Yeah, sure. So the Q1 roadmap is mostly set. Uh, we have two things still kinda hanging — the onboarding revamp and the advanced analytics pilot. I need clarity if onboa



In [3]:
if transcript_text:
    print("🔮 Step 2: Analyzing with Gemini...")
    api_key = os.getenv('GOOGLE_API_KEY')
    
    if not api_key:
        print("❌ ERROR: GOOGLE_API_KEY not found in .env file")
        print("💡 Create a .env file with: GOOGLE_API_KEY=your_actual_key")
    else:
        try:
            analysis_results = analyze_meeting(transcript_text, api_key=api_key)
            
            # Print results
            print_analysis_results(analysis_results)
            
            # Save results
            import json
            with open('../assets/example_task_output.json', 'w') as f:
                json.dump(analysis_results, f, indent=2)
            print(f"\n💾 Results saved to assets/example_task_output.json")
            
        except Exception as e:
            print(f"❌ Error in analysis: {e}")
else:
    print("❌ Cannot proceed - no transcript loaded")

🔮 Step 2: Analyzing with Gemini...
📊 MEETING ANALYSIS RESULTS

📋 MEETING SUMMARY:
   The meeting focused on prioritizing the onboarding revamp for Q1 and pushing the advanced analytics pilot to March. Key outcomes included decisions for a silent onboarding launch, modular build, and numerous tasks assigned to prepare for an internal demo on Thursday, improve user experience, and manage internal expectations.

✅ KEY DECISIONS (5):
   1. Onboarding revamp is the top priority for Q1.
   2. The advanced analytics pilot is officially pushed to March.
   3. The onboarding launch will be a silent update with no external marketing hype (no ads or blog posts).
   4. Prioritize core onboarding steps first; extra features can wait.
   5. Onboarding should be built as modular pieces to allow for easier future iterations and avoid rewrites.

👥 PARTICIPANTS (5):
   • Jeffy
   • Mira
   • Arun
   • Leah
   • Daniel

🎯 ACTIONABLE TASKS (31):
----------------------------------------

1. Update roadmap 

In [4]:
# Step 3: Validation & Planning
print("\n" + "="*60)
print("Step 3: Validation & Planning")
print("="*60)

from src.validate import validate_tasks, deduplicate_tasks
from src.planner import plan_tasks

# Get tasks and participants from previous analysis
tasks = analysis_results.get('tasks', [])
participants = analysis_results.get('participants', [])

print(f"📋 Original tasks: {len(tasks)}")

# Validate tasks
validated_tasks = validate_tasks(tasks, participants)
valid_owners = sum(1 for task in validated_tasks if task.get('owner_valid', False))
print(f"✅ Valid owners: {valid_owners}/{len(validated_tasks)}")

# Deduplicate
deduplicated_tasks = deduplicate_tasks(validated_tasks)
print(f"🔄 After deduplication: {len(deduplicated_tasks)} tasks")

# Add execution plans
planned_tasks = plan_tasks(deduplicated_tasks)

# Show sample planned task
if planned_tasks:
    sample_task = planned_tasks[0]
    print(f"\n📝 Sample Planned Task:")
    print(f"   Title: {sample_task.get('title')}")
    print(f"   Owner: {sample_task.get('owner')} (Valid: {sample_task.get('owner_valid', False)})")
    print(f"   Effort: {sample_task.get('estimated_effort')}")
    print(f"   Steps: {sample_task.get('execution_steps', [])}")

# Update results with planned tasks
analysis_results['planned_tasks'] = planned_tasks

# Save updated results
import json
with open('../assets/example_task_output.json', 'w') as f:
    json.dump(analysis_results, f, indent=2)
print(f"\n💾 Updated results saved!")


Step 3: Validation & Planning
📋 Original tasks: 31
✅ Valid owners: 31/31
🔄 After deduplication: 31 tasks

📝 Sample Planned Task:
   Title: Update roadmap for Q1
   Owner: Mira (Valid: True)
   Effort: Small
   Steps: ['Assess current state', 'Plan changes', 'Implement updates', 'Test and verify']

💾 Updated results saved!


In [6]:

print("\n" + "="*60)
print("Step 4: Action & Dashboard")
print("="*60)


# Get the planned tasks
planned_tasks = analysis_results.get('planned_tasks', [])
meeting_summary = analysis_results.get('meeting_summary', '')
participants = analysis_results.get('participants', [])

# Display summary
display_task_summary(planned_tasks)

# Generate follow-up email
email_subject, email_body = generate_followup_email(meeting_summary, planned_tasks, participants)
print(f"\n📧 FOLLOW-UP EMAIL DRAFT:")
print("=" * 50)
print(f"Subject: {email_subject}")
print(email_body[:500] + "..." if len(email_body) > 500 else email_body)
print("=" * 50)

# Export to files
export_to_csv(planned_tasks, '../assets/tasks_export.csv')
export_to_json(planned_tasks, '../assets/tasks_export.json')
print(f"📊 Exports: assets/tasks_export.csv, assets/tasks_export.json")

# Generate HTML dashboard
html_dashboard = generate_task_dashboard(planned_tasks)
with open('../assets/tasks_dashboard.html', 'w', encoding='utf-8') as f:
    f.write(html_dashboard)
print(f"🌐 HTML dashboard: assets/tasks_dashboard.html")

# Save final comprehensive results
final_output = {
    'metadata': {
        'generated_at': datetime.now().isoformat(),
        'meeting_file': 'meeting_01.txt',
        'total_tasks': len(planned_tasks)
    },
    'analysis_results': analysis_results
}

with open('../assets/final_output.json', 'w') as f:
    json.dump(final_output, f, indent=2)
print(f"💾 Final results: assets/final_output.json")

print("\n🎉 PIPELINE COMPLETE! 🎉")
print("You now have a full Meeting→Execution pipeline!")


Step 4: Action & Dashboard

📊 TASK SUMMARY:
Total Tasks: 31

📈 By Priority:
   High: 20
   Medium: 10
   Low: 1

⚡ By Effort:
   Small: 9
   Large: 19
   Medium: 3

👥 By Owner:
   Mira: 12
   Leah: 10
   Arun: 6
   Daniel: 3

📧 FOLLOW-UP EMAIL DRAFT:
Subject: Action Items from Meeting - 2025-11-27
Hi team,

Here are the action items from our recent meeting:

MEETING SUMMARY:
The meeting focused on prioritizing the onboarding revamp for Q1 and pushing the advanced analytics pilot to March. Key outcomes included decisions for a silent onboarding launch, modular build, and numerous tasks assigned to prepare for an internal demo on Thursday, improve user experience, and manage internal expectations.

ACTION ITEMS:

1. Update roadmap for Q1
   • Owner: Mira
   • Deadline: TBD
   • Priority: Hi...
📊 Exports: assets/tasks_export.csv, assets/tasks_export.json
🌐 HTML dashboard: assets/tasks_dashboard.html
💾 Final results: assets/final_output.json

🎉 PIPELINE COMPLETE! 🎉
You now have a full Mee

In [10]:
# Step 5: Process ALL Meetings
print("\n" + "="*60)
print("Step 5: Process ALL Meetings for Proper Evaluation")
print("="*60)


# List all meeting files
meeting_files = [
    'meeting_01.txt', 'meeting_02.txt', 'meeting_03.txt', 
    'meeting_04.txt', 'meeting_05.txt'
]

# Create results directory
os.makedirs('../assets/batch_results', exist_ok=True)

all_meeting_results = {}

for meeting_file in meeting_files:
    print(f"\n🔄 Processing {meeting_file}...")
    
    try:
        # Step 1: Ingest
        transcript_path = f'../data/sample_transcripts/{meeting_file}'
        transcript_text = ingest.process_transcript(transcript_path)
        
        # Step 2: Analyze with Gemini
        analysis_results = analyze_meeting(transcript_text)
        
        # Step 3: Validate & Plan
        tasks = analysis_results.get('tasks', [])
        participants = analysis_results.get('participants', [])
        
        validated_tasks = validate_tasks(tasks, participants)
        deduplicated_tasks = deduplicate_tasks(validated_tasks)
        planned_tasks = plan_tasks(deduplicated_tasks)
        
        # Update results with planned tasks
        analysis_results['planned_tasks'] = planned_tasks
        
        # Save individual meeting results
        meeting_id = meeting_file.replace('.txt', '')
        output_path = f'../assets/batch_results/{meeting_id}_output.json'
        
        final_output = {
            'metadata': {
                'generated_at': datetime.now().isoformat(),
                'meeting_file': meeting_file,
                'total_tasks': len(planned_tasks)
            },
            'analysis_results': analysis_results
        }
        
        with open(output_path, 'w') as f:
            json.dump(final_output, f, indent=2)
        
        all_meeting_results[meeting_id] = final_output
        print(f"✅ {meeting_file}: {len(planned_tasks)} tasks saved to {output_path}")
        
    except Exception as e:
        print(f"❌ Error processing {meeting_file}: {e}")

# Save combined results
with open('../assets/all_meetings_output.json', 'w') as f:
    json.dump(all_meeting_results, f, indent=2)

print(f"\n🎉 BATCH PROCESSING COMPLETE!")
print(f"📁 Results saved to assets/batch_results/")


Step 5: Process ALL Meetings for Proper Evaluation

🔄 Processing meeting_01.txt...
✅ meeting_01.txt: 24 tasks saved to ../assets/batch_results/meeting_01_output.json

🔄 Processing meeting_02.txt...
✅ meeting_02.txt: 14 tasks saved to ../assets/batch_results/meeting_02_output.json

🔄 Processing meeting_03.txt...
✅ meeting_03.txt: 17 tasks saved to ../assets/batch_results/meeting_03_output.json

🔄 Processing meeting_04.txt...
✅ meeting_04.txt: 10 tasks saved to ../assets/batch_results/meeting_04_output.json

🔄 Processing meeting_05.txt...
✅ meeting_05.txt: 10 tasks saved to ../assets/batch_results/meeting_05_output.json

🎉 BATCH PROCESSING COMPLETE!
📁 Results saved to assets/batch_results/


In [2]:
# Step 6: Proper Evaluation with All Meetings
print("\n" + "="*60)
print("Step 6: Proper Evaluation with All Meetings")
print("="*60)

from src.evaluate import run_comprehensive_evaluation, print_evaluation_summary

# Run evaluation using batch results
evaluation_results = run_comprehensive_evaluation(
    ground_truth_path='../data/annotations/ground_truth.csv',
    ai_outputs_dir='../assets/batch_results'
)

# Print summary
print_evaluation_summary(evaluation_results)

# Save evaluation results
with open('../assets/proper_evaluation_results.json', 'w') as f:
    json.dump(evaluation_results, f, indent=2)
print(f"\n💾 Proper evaluation results saved!")


Step 6: Proper Evaluation with All Meetings
🔍 RUNNING COMPREHENSIVE EVALUATION
✅ Loaded ground truth: 76 tasks from 5 meetings
📊 Ground truth columns: ['transcript_id', 'task_id', 'task_title', 'task_description', 'owner', 'owner_email', 'deadline', 'priority', 'evidence_start', 'evidence_text']
📊 Evaluating 5 meetings: ['meeting_01', 'meeting_02', 'meeting_03', 'meeting_04', 'meeting_05']

📋 Evaluating meeting_01...
✅ Loaded AI predictions: 24 tasks
   ✅ Tasks: GT=19, AI=24
   📈 F1: 0.744, Precision: 0.667, Recall: 0.842
   👤 Owner Accuracy: 0.421
   ⚡ Priority Accuracy: 0.316

📋 Evaluating meeting_02...
✅ Loaded AI predictions: 14 tasks
   ✅ Tasks: GT=14, AI=14
   📈 F1: 0.643, Precision: 0.643, Recall: 0.643
   👤 Owner Accuracy: 0.571
   ⚡ Priority Accuracy: 0.714

📋 Evaluating meeting_03...
✅ Loaded AI predictions: 17 tasks
   ✅ Tasks: GT=18, AI=17
   📈 F1: 0.686, Precision: 0.706, Recall: 0.667
   👤 Owner Accuracy: 0.412
   ⚡ Priority Accuracy: 0.529

📋 Evaluating meeting_04...
✅ 